In [1]:
import pandas as pd
import numpy as np

In [9]:
df_feature = pd.read_csv("Features data set.csv")
df_sales = pd.read_csv("sales data-set.csv")

In [7]:
df_feature[df_feature['Store'] == 1]

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
...,...,...,...,...,...,...,...,...,...,...,...,...
177,1,28/06/2013,83.94,3.495,4205.98,796.70,6.84,3816.78,4812.74,NaN,NaN,False
178,1,05/07/2013,79.85,3.422,7649.99,3503.29,1766.77,9454.96,1079.89,NaN,NaN,False
179,1,12/07/2013,83.12,3.400,6089.94,1362.42,209.62,2367.42,2651.05,NaN,NaN,False
180,1,19/07/2013,79.26,3.556,3117.04,1060.39,199.05,1012.30,5381.72,NaN,NaN,False


In [8]:
df_sales[df_sales['Store'] == 1]

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False
...,...,...,...,...,...
10239,1,99,31/08/2012,20.06,False
10240,1,99,07/09/2012,0.05,True
10241,1,99,14/09/2012,0.03,False
10242,1,99,05/10/2012,635.00,False


In [10]:
df_sales.Date = pd.to_datetime(df_sales.Date)
df_feature.Date = pd.to_datetime(df_feature.Date)

df_sales = df_sales.sort_values('Date')
df_feature = df_feature.sort_values('Date')



In [11]:
df_feature = df_feature[df_feature.Date.dt.date <= df_sales.Date.dt.date.max()]

In [15]:
df = df_sales.merge(df_feature, on = ['Store','Date'], how = 'left')
df = df.dropna()


,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
158277,1,20,2011-02-12,3117.17,False,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,218.714733,7.866,False
158278,13,17,2011-02-12,22202.30,False,33.94,3.389,7945.42,56.00,25126.90,5550.87,31844.20,129.845967,6.392,False
158279,5,56,2011-02-12,698.75,False,51.14,3.172,2359.44,4.00,658.01,901.67,24751.93,219.291729,6.300,False
158280,13,97,2011-02-12,22007.05,False,33.94,3.389,7945.42,56.00,25126.90,5550.87,31844.20,129.845967,6.392,False
158282,40,87,2011-02-12,23437.96,False,38.51,3.501,3966.04,23.47,2363.99,1455.15,15828.62,136.490933,4.420,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418557,13,24,2012-11-05,7901.86,False,56.47,3.809,14566.92,86.00,60.34,6009.18,4224.91,131.145333,5.965,False
418562,13,23,2012-11-05,27157.91,False,56.47,3.809,14566.92,86.00,60.34,6009.18,4224.91,131.145333,5.965,False
418566,4,98,2012-11-05,15398.45,False,67.64,3.685,20499.88,86.00,103.05,5005.96,3815.16,131.145333,4.308,False
418574,13,40,2012-11-05,80704.69,False,56.47,3.809,14566.92,86.00,60.34,6009.18,4224.91,131.145333,5.965,False
